# Mincut/Maxcut problem

A network is represent by a set of links shown below. Each vertex represents a person that the government can bribe (in the way mentioned in the lecture video). The price to bribe each person is also shown in the data below. 

Find the set of person to bribe such that the government gets the maximum amount of secret data. Then calculate the total price needed to bribe those people.

Solve this problem using ILP and, if you have an access to Gurobi solver, using QUBO.

In [6]:
bribe_price = {0: 12,
            1: 10,
            2: 5,
            3: 4,
            4: 10,
            5: 20,
            6: 10,
            7: 5,
            8: 4,
            9: 10,
            10: 20,
            11: 10,
            12: 5,
            13: 4,
            14: 10,
            15: 20
}

link = [(0,1),
        (0,4),
        (1,5),
        (1,7),
        (1,8),
        (1,9),
        (1,14),
        (1,15),
        (2,3),
        (3,4),
        (4,6),
        (4,7),
        (4,8),
        (4,11),
        (4,13),
        (4,15),
        (5,6),
        (5,15),
        (6,7),
        (6,8),
        (7,10),
        (7,11),
        (7,14),
        (8,10),
        (9,13),
        (10,11),
        (10,12),
        (10,13),
        (10,14),
        (10,15),
        (11,12),
        (12,15),
        (13,14),
        (14,15)
        ]

In [7]:
# using ILP
from pulp import LpMaximize, LpMinimize, LpStatus, lpSum, LpVariable, LpProblem
import pulp
solver_list=pulp.listSolvers(onlyAvailable=True)
print(solver_list)

No parameters matching '_test' found
['GUROBI', 'PULP_CBC_CMD']


In [8]:
model=LpProblem(name='mincut_maxcut_problem', sense=LpMinimize)
#model=LpProblem(name='mincut_maxcut_problem', sense=LpMaximize)

In [9]:
x={i: LpVariable(name=f'x{i}', cat="Binary") for i in range(len(bribe_price))}
y={(link[i][0], link[i][1], j): LpVariable(name=f'y({link[i][0]},{link[i][1]})_{j}', cat="Binary") for i in range(len(link)) for j in range(0, 3)}
# y{u,v}0=neither, y{u,v}1=either, y{u,v}2=both
#y

In [10]:
# objective function
#model+=lpSum(x[i] for i in range(len(bribe_price)))  # not work!
#model+=lpSum(x[i]*bribe_price[i] for i in range(len(bribe_price)))

#model+=lpSum(y[(link[i][0], link[i][1], 1)]for i in range(len(link)))  # try to maximize bribe 1 node in each edge
model+=lpSum(y[(link[i][0], link[i][1], 0)]+y[(link[i][0], link[i][1], 2)] for i in range(len(link))) # minize bribe 0 & 2 nodes in each edge

In [11]:
# constraints
for i in range(len(link)):
    # Ye,0 + Ye,1 + Ye,2 = 1 for e in Edge
    # 1 when Y1=1, 2 when Y2=1, 0 when Y1&Y2=0
    model+=y[(link[i][0], link[i][1], 0)]+y[(link[i][0], link[i][1], 1)]+y[(link[i][0], link[i][1], 2)] == 1
    
    # Xu + Xv = Ye,1 + 2*Ye,2 for {u,v} in Edge
    model+=x[link[i][0]]+x[link[i][1]] == y[(link[i][0], link[i][1], 1)]+2*y[(link[i][0], link[i][1], 2)]
    
    #model+=x[link[i][0]]+x[link[i][1]] >= 1


# cheat (ans=7)!
#model+=lpSum(x[i] for i in range(len(bribe_price))) == 7
# maximum secret data!
#model+=lpSum(y[(link[i][0], link[i][1], 1)]+y[(link[i][0], link[i][1], 2)] for i in range(len(link))) == len(link)

In [12]:
status=model.solve()
status

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/panithi/.local/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/ff022cd6ec134bfa904a9bfeb8b95f96-pulp.mps timeMode elapsed branch printingOptions all solution /tmp/ff022cd6ec134bfa904a9bfeb8b95f96-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 73 COLUMNS
At line 616 RHS
At line 685 BOUNDS
At line 804 ENDATA
Problem MODEL has 68 rows, 118 columns and 238 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0 - 0.00 seconds
Cgl0004I processed model has 68 rows, 117 columns (117 integer (117 of which binary)) and 237 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 15 integers unsatisfied sum - 7.5
Cbc0038I Pass   1: suminf.    7.50000 (15) obj. 0 iterations 0
Cbc0038I Pass   2: suminf.    7.50000 (15) obj. 0 iterations 0
Cbc0038I Pass   3: s

1

In [13]:
model.objective.value()

7.0

In [16]:
cnt_person=0
total_price=0
for var in model.variables():
    if var.value()==0 and var.name[0]=='x': # ILP not choose -> but we choose this least common group instead
        print(f'{var.name}: {var.value()} [{bribe_price[int(var.name[1:])]} ฿]')
        cnt_person+=1
        total_price+=bribe_price[int(var.name[1:])]
    #if var.value()==1 and var.name[0]=='y': # selected edge
    #    print(f'{var.name}: {var.value()}')

print("Number of person to bribe is", cnt_person)
print("Total price is", total_price)

x1: 0.0 [10 ฿]
x10: 0.0 [20 ฿]
x12: 0.0 [5 ฿]
x14: 0.0 [10 ฿]
x2: 0.0 [5 ฿]
x4: 0.0 [10 ฿]
x6: 0.0 [10 ฿]
Number of person to bribe is 7
Total price is 70


In [17]:
model

mincut_maxcut_problem:
MINIMIZE
1*y(0,1)_0 + 1*y(0,1)_2 + 1*y(0,4)_0 + 1*y(0,4)_2 + 1*y(1,14)_0 + 1*y(1,14)_2 + 1*y(1,15)_0 + 1*y(1,15)_2 + 1*y(1,5)_0 + 1*y(1,5)_2 + 1*y(1,7)_0 + 1*y(1,7)_2 + 1*y(1,8)_0 + 1*y(1,8)_2 + 1*y(1,9)_0 + 1*y(1,9)_2 + 1*y(10,11)_0 + 1*y(10,11)_2 + 1*y(10,12)_0 + 1*y(10,12)_2 + 1*y(10,13)_0 + 1*y(10,13)_2 + 1*y(10,14)_0 + 1*y(10,14)_2 + 1*y(10,15)_0 + 1*y(10,15)_2 + 1*y(11,12)_0 + 1*y(11,12)_2 + 1*y(12,15)_0 + 1*y(12,15)_2 + 1*y(13,14)_0 + 1*y(13,14)_2 + 1*y(14,15)_0 + 1*y(14,15)_2 + 1*y(2,3)_0 + 1*y(2,3)_2 + 1*y(3,4)_0 + 1*y(3,4)_2 + 1*y(4,11)_0 + 1*y(4,11)_2 + 1*y(4,13)_0 + 1*y(4,13)_2 + 1*y(4,15)_0 + 1*y(4,15)_2 + 1*y(4,6)_0 + 1*y(4,6)_2 + 1*y(4,7)_0 + 1*y(4,7)_2 + 1*y(4,8)_0 + 1*y(4,8)_2 + 1*y(5,15)_0 + 1*y(5,15)_2 + 1*y(5,6)_0 + 1*y(5,6)_2 + 1*y(6,7)_0 + 1*y(6,7)_2 + 1*y(6,8)_0 + 1*y(6,8)_2 + 1*y(7,10)_0 + 1*y(7,10)_2 + 1*y(7,11)_0 + 1*y(7,11)_2 + 1*y(7,14)_0 + 1*y(7,14)_2 + 1*y(8,10)_0 + 1*y(8,10)_2 + 1*y(9,13)_0 + 1*y(9,13)_2 + 0
SUBJECT TO
_C1: y(0,1)_0